In [ ]:
from Common import utils
from Robot import kdl_utils
import numpy as np
import tf_conversions

#### Error recovery

In [ ]:
!rostopic pub -1 /franka_ros_interface/franka_control/error_recovery/goal franka_msgs/ErrorRecoveryActionGoal "{}"

#### Convert teleop recorded data to get bottelneck poses

##### for 100bw

In [ ]:
weights = [0, 0, 0, 0, 1]

#### weighted B-pose

In [ ]:
denomin = 2+5+20+50+100
weights = [2/denomin, 5/denomin, 20/denomin, 50/denomin, 100/denomin]

##### Based on prev *weights* save data for Coarse-to-fine self-learning

In [ ]:
data_folders = np.arange(1,23,1)

for folder in data_folders:
    teleopp_data_path = '/home/prashanth/catkin_ws/src/tele_operation/scripts/data/' + str(folder)
    filenames = ["/traj_B_bw2.npy", "/traj_B_bw5.npy", "/traj_B_bw20.npy", "/traj_B_bw50.npy", "/traj_B_bw100.npy"]

    poses = np.array([np.load(teleopp_data_path+fname, allow_pickle=True)[0] for fname in filenames])

    pos_ori = []
    for i, pose in enumerate(poses):
        bottleneck_pose = tf_conversions.fromMsg(pose)
        pos = bottleneck_pose.p
        ori = bottleneck_pose.M.GetQuaternion()
        pos_ori.append([*pos,  *ori])

    pos_ori = np.array(pos_ori)

    
    # position weighted
    pos_w = sum(w * p for p, w in zip(pos_ori[:,:3], weights))
    # orientation weighted
    A = sum(w * (np.outer(q, q)) for q, w in zip(pos_ori[:,3:], weights))
    ori_w = np.linalg.eigh(A)[1][:, -1]

    weighted_b_pose = tf_conversions.fromTf((pos_w, ori_w))
    weighted_b_pose_msg = tf_conversions.toMsg(weighted_b_pose)

    if(weighted_b_pose_msg.position.z < 0.11):
        weighted_b_pose_msg.position.z = 0.11

    # Change toi the name of the path u need the data to be saved to
    task_name = str(folder)+"_W" + '/Traj_20'

    bottleneck_pose = tf_conversions.fromMsg(weighted_b_pose_msg)
    # Then create the vertical bottleneck pose
    bottleneck_pose_vertical = kdl_utils.create_vertical_pose_from_x_y_z_theta(bottleneck_pose.p[0], bottleneck_pose.p[1], bottleneck_pose.p[2], bottleneck_pose.M.GetRPY()[2])
    # And then create the transformation, in end-effector frame, between the vertical and demonstration bottleneck
    bottleneck_transformation_vertical_to_demo = bottleneck_pose_vertical.Inverse() * bottleneck_pose

    print('Saving data ...', task_name)
    data_directory = '../Data/' + str(task_name) + '/Single_Demo/Raw'
    utils.create_or_clear_directory(data_directory)
    bottleneck_pose_vector = kdl_utils.create_vector_from_pose(bottleneck_pose)
    np.save(data_directory + '/bottleneck_pose_vector.npy', bottleneck_pose_vector)
    bottleneck_pose_vector_vertical = kdl_utils.create_vector_from_pose(bottleneck_pose_vertical)
    np.save(data_directory + '/bottleneck_pose_vector_vertical.npy', bottleneck_pose_vector_vertical)

    bottleneck_transformation_vector = kdl_utils.create_vector_from_pose(bottleneck_transformation_vertical_to_demo)
    transformation_path = data_directory + '/bottleneck_transformation_vector_vertical_to_demo.npy'
    np.save(transformation_path, bottleneck_transformation_vector)

##### Looped Dataset creation from saved images and poses for users

In [ ]:
from Dataset_Creation.image_to_pose_dataset_creator_coarse import ImageToPoseDatasetCreatorCoarse
from Dataset_Creation.image_to_pose_dataset_creator_fine import ImageToPoseDatasetCreatorFine

data_folders = np.arange(1,11,1)

for folder in data_folders:
    # For weighted BW
    task_name = str(folder)+"_W" + '/Traj_20'
    dataset_creator_coarse = ImageToPoseDatasetCreatorCoarse(task_name=task_name)
    dataset_creator_coarse.run()
    dataset_creator_fine = ImageToPoseDatasetCreatorFine(task_name=task_name)
    dataset_creator_fine.run()

    # For highest BW
    task_name = str(folder) + '/Traj_20'
    dataset_creator_coarse = ImageToPoseDatasetCreatorCoarse(task_name=task_name)
    dataset_creator_coarse.run()
    dataset_creator_fine = ImageToPoseDatasetCreatorFine(task_name=task_name)
    dataset_creator_fine.run()

#### Train networks on datasets

In [ ]:
from Training.image_to_pose_trainer_coarse import ImageToPoseTrainerCoarse
from Training.pose_to_uncertainty_trainer import PoseToUncertaintyTrainer
from Training.image_to_pose_trainer_fine import ImageToPoseTrainerFine
from Common import config

data_folders = np.arange(8,11,1)

for folder in data_folders:
    task_name = str(folder) +"_W" + '/Traj_20'
    # Train Coarse network
    num_trajectories = config.NO_OF_TRAJECTORIES
    trainer = ImageToPoseTrainerCoarse(task_name=task_name, num_trajectories=num_trajectories)
    trainer.train()
    trainer = PoseToUncertaintyTrainer(task_name=task_name, num_trajectories=num_trajectories)
    trainer.run()

    # Train fine network
    num_trajectories = config.NO_OF_TRAJECTORIES
    trainer = ImageToPoseTrainerFine(task_name=task_name, num_trajectories=num_trajectories)
    trainer.train()

#### Testing the laerned network

In [ ]:
from Testing.coarse_tester import CoarseTester

data_folders = np.arange(10,11,1)

for folder in data_folders:
    task_name = str(folder) +"_W" + '/Traj_20'
    estimation_method = 'filtering_with_static_uncertainty'
    tester = CoarseTester()
    tester.run(task_name, estimation_method=estimation_method)

#### Robot

In [ ]:
from Robot.sawyer import Sawyer
import rospy

rospy.init_node('notebook_node')
robot = Sawyer()


In [ ]:
robot.robot.move_to_neutral()

In [ ]:
import PyKDL
bottleneck_pose_vector = np.load('../Data/3/Traj_20/Single_Demo/Raw/bottleneck_pose_vector.npy')
bottleneck_pose = kdl_utils.create_pose_from_vector(bottleneck_pose_vector)
print(tf_conversions.toMsg(bottleneck_pose))
robot.move_to_pose(bottleneck_pose)